<img src='https://upload.wikimedia.org/wikipedia/fr/thumb/e/ed/Logo_Universit%C3%A9_du_Maine.svg/1280px-Logo_Universit%C3%A9_du_Maine.svg.png' width="300" height="500">
<br>
<div style="border: solid 3px #000;">
    <h1 style="text-align: center; color:#000; font-family:Georgia; font-size:26px;">Infrastructures pour l'IA</h1>
    <p style='text-align: center;'>Master Informatique 1</p>
    <p style='text-align: center;'>Anhony Larcher</p>
</div>

Cette session est organisée comme un challenge:
* Vous optimiserez un code afin d'effectuer un calcul le plus rapidement possible
* le résultat sera utilisée dans le TP numéro 6 pour faire du calcul parallélisé.


# Implémenter le calcul suivant et optimisez le au mieux

Il s'agit d'accumuler les statistiques d'ordre 0 et d'ordre 1 sur un mélange de Gaussiennes. Vous trouverez sur Umtice une archive zip contenant un objet Mixture et des paramètres acoustiques de type MFCC.

Chaque distribution de la mixture a pour densité de probabilité:

Et la densité de probabilité du mélange de Gaussienne est 
$$p(x|\lambda) = \Sigma_{i=1}^{M}w_ip_i(x)$$

où $$\Sigma_{i=1}^{M}w_i = 1$$

et 

$$p_i(x) = \frac{1}{(2\pi)^{\frac{D}{2}}|\Sigma_i|^\frac{1}{2}} e^{-(\frac{1}{2}(x - \mu)^T\Sigma_i^{-1}(x-\mu))}$$

Nous souhaiton ici calculer pour chaque trame $x_t$

$$P(i|x_t) = \frac{w_ip_i(x_t)}{\Sigma_{j=1}^M w_j p_j(x_t)}$$

Pour ensuite calculer les statistiques d'ordre 0 pour chaque gaussienne $i$:

$$n_i = \Sigma_{t=1}^{T}P(i|x_t)$$

Les statistiques d'ordre 1:

$$E_i(x) = \frac{1}{n_i} \Sigma_{t=1}^{T}P(i|x_t)x_t$$

Et les statistiques d'ordre 2:

$$E_i(x^2) = \frac{1}{n_i} \Sigma_{t=1}^{T}P(i|x_t)x_t^2$$

Les données sont disponibles ici:

    https://umbox.univ-lemans.fr/index.php/s/CiP47cfw8NBMM5J

## 1.1) Initialisez l'accumulateur de statistiques

Le fichier ``mixture.py``contient le code d'une classe Mixtuyre qui sera utilisé dans ce TP. Ce code se trouve également dans la cellule ci-dessous pour travailler dans le notebook. Une fois la cellule ci-dessous complétée vous pourrez recopier le code dans le fichier ``mixture.py``afin d'avoir une version propre du code.

In [2]:
import numpy
import pickle

class Mixture(object):

    def __init__(self):
        """
        Initialize an empty Mixture object
        """
        self.w = numpy.array([])
        self.mu = numpy.array([])
        self.invcov = numpy.array([])
        self.cst = numpy.array([])
        self.det = numpy.array([])
        self.D = 0 

    def __repr__(self):
        """
        Serialize a Mixture object to text
        """
        return f'w = {self.w.shape}{self.w}\nmu = {self.mu.shape}{self.mu}\ninvcov = {self.invcov.shape}{self.invcov}\ncst = {self.cst.shape}{self.cst}\ndet = {self.det.shape}{self.det}'

    @classmethod
    def read(cls, filename):
        """
        Read a Mixture object stored in Pickle format on disk
        :param filename: the name of the file to read from
        :return: a Mixture object
        """
        with open(filename, 'rb') as fh:
            mixture = pickle.load(fh)
            return mixture

    def save(self, filename):
        """
        Save a Mixture object to disk in Pickle format
        :param filename: the name of the file to write to
        """
        with open(filename, 'wb') as fh:
            pickle.dump(self, fh)

    def loadPresets(self):
        self.mu = numpy.load("mu.npy")
        self.w  = numpy.load("w.npy")
        self.invcov = numpy.load("invcov.npy")
        self.D = self.mu.shape[1]


## Analyse des formules

On remarque d'abord que certains éléments dépendent des données, et plus particulièrement de chaque vecteur de donnée, mais également de chaque distribution de la mixture.

Nous verrons donc apparaitre 2 boucles principales:
* une sur les données
* une sur les distributions

Pour simplifier le calcul nous devons donc séparer ce qui est indépendant de ces boucles afin de ne pas le recalculer plusieurs fois.

Ré-écrivez $$p_i(x)$$ en séparant ces termes.

$$p_i(x) = \frac{1}{(2\pi)^{\frac{D}{2}}|\Sigma_i|^\frac{1}{2}} e^{-(\frac{1}{2}(x - \mu_i)^T\Sigma_i^{-1}(x-\mu_i))}$$

On trouve d'abord un terme qui dépend de chaque mixture: leur déterminant

In [6]:
# Il faut donc calculer ce déterminant une seule fois:
# Les covariances inverses étant stockées par ligne dans self.invcov nous calculons simplement le produit de chaque ligne.
# Le grand Sigma dans notre équation, c'est la matrice de covariance.
# Comme on l'a expliqué en TD, pour simplifier les calculs on ne prend qu'une matrice diagonale, car ça va plus vite à calculer
# Ce qui est chouette c'est que le determinant d'une matrice diagonale est juste le produit de ses termes diagonaux.
# Seul petit détail, on nous a donné l'inverse de la covariance. Il faudra donc la recalculer.
# C'est pas bien gênant, on fait juste 1/v pour chaque valeur, car dans le cas d'une matrice diagonale on peut calculer son inverse comme ça.
#
# Just a little note, in the following code cells of this notebook, I am not going to use the mixture object above, but rather do the code inline and adapt it in the Mixture class.

from time import perf_counter_ns as perf

invcov = numpy.load("invcov.npy")

# With loops and no numpy
start = perf()
det = []
for line in invcov:
    det.append(1)
    for val in line:
        det[-1] *= val
    det[-1] = 1 / det[-1]
end = perf()
naive_imple = end-start
        
start = perf()
det = 1.0 / numpy.prod(invcov, axis=1)
end = perf()
numpy_imple = end-start

print(f"Naive implementation took {naive_imple/1000} microseconds\n"
      f"Numpy implementation took {numpy_imple/1000} microseconds\n"
      f"So, numpy here is {naive_imple/numpy_imple:.2f} times faster")

Naive implementation took 1712.3 nanoseconds
Numpy implementation took 211.3 nanoseconds
So, numpy here is 8.10 times faster


Nous pouvons maintenant calculer le premier terme pour chaque distribution:
    
$$ \frac{1}{(2\pi)^{\frac{D}{2}}|\Sigma_i|^\frac{1}{2}} $$ 

In [7]:
# D étant constant, on voit d'ores et déjà que (2Pi)^(D/2) est une constante également. Le déterminant de la matrice de covariance ne dépend pas des donnés, on peut donc calculer cette fraction en amont, une seule fois pour toutes les données puisqu'elles n'influent pas dans le calcul ici.
# Enfin, n'oublions la racine carré sur le determinant de la covariance.
# Manque plus qu'inverser tout ça et ce sera bon.

import math

D = invcov.shape[1]

### Pure python implementation ###
start = perf()
cst = []
for line in det:
    cst.append( 1.0 / (2.0 * math.pi)**(D/2.0) * (math.sqrt(line)) )
end = perf()
naive_imple = end-start

### Numpy implementation ###
start = perf()
cst = 1.0 / ( numpy.power(2.0 * numpy.pi, D/2.0) * numpy.sqrt( det ))
end = perf()
numpy_imple = end-start

print(f"Naive implementation took {naive_imple/1000} microseconds\n"
      f"Numpy implementation took {numpy_imple/1000} microseconds\n"
      f"So, numpy here is {naive_imple/numpy_imple:.2f} times faster")


Naive implementation took 332.8 microseconds
Numpy implementation took 256.2 microseconds
So, numpy here is 1.30 times faster


Regardons maintenant le terme contenu dans l'exponentielle:

$$ -(\frac{1}{2}(x - \mu_i)^T\Sigma_i^{-1}(x-\mu_i)) $$ 

Tout ce terme dépend des distributions, une partie seulement dépend des données.

$$ -(\frac{1}{2}(x - \mu_i)^T\Sigma_i^{-1}(x-\mu_i)) = -\frac{1}{2} ( x^T\Sigma_i^{-1}x + \mu_i^T\Sigma_i^{-1}\mu_i - x^T\Sigma_i^{-1}\mu_i - \mu_i^T\Sigma_i^{-1}x ) $$ 

Calculez chacun de ces termes:

Analyse du terme independant des données:

$$ \mu_i^T\Sigma_i^{-1}\mu_i $$

$$ \mu_i^T$$ est de dimension (1, 39) $$\Sigma_i$$ est de dimension (39, 39)

Donc $$ \mu_i^T\Sigma_i^{-1}\mu_i $$ est de dimension (1,)

In [4]:
# En pratique, comme la matruice de covariance est diagonale, nous n'avons pas besoin de calculer un vrai produit matriciel
# Écrivez la valeur du premier terme de \mu_i \Sigma^{-1}_i et tirez partie de cette expression pour simplifier
# le calcul de ce terme

# self.mu est de dimension 64x39
# self.invcov est de dimension 64x39
# numpy.square(self.mu) * self.invcov) est de dimension 64x39
# numpy.square(self.mu) * self.invcov).sum(1) est de dimension 64 
# A est de dimension 64

a_idpt = numpy.sum( numpy.square(mixture.mu) * mixture.invcov, axis=1 )
print(a_idpt, a_idpt.shape)

[31.06258507 34.9974304  25.07955728 24.4477801  29.30450087 23.53093618
 17.97478978 27.49854067 33.14749497 24.18498924 21.74967277 20.35285252
 23.32292406 39.97481249 32.31881181 25.03008533 20.10366642 32.42881905
 32.56320818 24.59221355 20.88498159 24.38585148 16.69366773 35.32670674
 14.82171863 30.3776522  42.53213539 40.03429768 28.0882865  30.53235822
 24.96461008 29.58541189 37.57946244 28.75486441 24.63392124 28.06858742
 39.07440781 39.92516607 22.50515188 22.14158314 25.76064448 28.13461136
 20.41493897 19.16378518 34.64329639 19.50864393 33.01843831 29.65358247
 17.39356694 29.01589355 31.18336031 28.7124705  24.41168099 24.92588735
 19.18626158 29.05813811 30.12623722 29.41842831 33.84483729 30.3030583
 40.39943314 27.4758611  29.12646595 26.39505375] (64,)


Pareil pour le terme $$ x^T\Sigma_i^{-1}x = \sum_{n=1}^{39}{x^2 \sigma_n} $$

In [5]:
X = numpy.load("features/features_0.npy")

def f(x):
    return numpy.sum( numpy.square(x) * mixture.invcov, axis=1 )

b_x2 = []

for x in X:
    b_x2.append(f(x))
    
b_x2 = numpy.array(b_x2)
print(b_x2, b_x2.shape)

[[33.31230817 35.57492947 38.99114351 ... 34.15860092 39.43752767
  30.8941242 ]
 [51.15131914 60.61843294 62.29082171 ... 56.49160923 66.87870798
  53.06242306]
 [70.83919397 83.36414292 87.95483019 ... 80.93612559 91.63412588
  70.79893953]
 ...
 [49.78288519 58.71514122 62.6343546  ... 58.57760071 61.74498848
  51.66981218]
 [43.27393968 51.37402635 56.18306032 ... 54.05507456 54.49923773
  46.33599516]
 [56.29844509 61.20991474 64.47810812 ... 58.69314188 64.52144284
  51.81331524]] (3288, 64)


$$ - x^T\Sigma_i^{-1}\mu_i - \mu_i^T\Sigma_i^{-1}x = 2 \cdot \sum_{n=1}^{39}{x \sigma_n \mu_n} $$ 

In [6]:

def g(x):
    return ( numpy.sum(x * mixture.invcov * mixture.mu, axis=1) )

c_musigmax = []

for x in X:
    c_musigmax.append(g(x))
    
c_musigmax = numpy.array(c_musigmax)
print(c_musigmax, c_musigmax.shape)

[[ -1.58801514  -5.96381771  -4.83292118 ...  -0.79741204  -3.76346581
   -5.35207078]
 [ -2.53498114 -13.52977725  -4.09763207 ...  11.81686344  12.40727693
   -8.18419267]
 [  7.57194635  16.00643795   4.36083431 ...   0.91937211   9.4549122
    3.3095916 ]
 ...
 [ -5.45560549   4.39783807  26.79397798 ...  10.33471163   5.1771689
    6.51704925]
 [ -3.02075171  10.48196304   3.28941427 ...  -2.26414699  -0.61125185
   -5.1614433 ]
 [ -2.543049     4.34422535   5.75279854 ...  -2.24565835  14.18855217
   -1.76493032]] (3288, 64)


In [7]:

exp = - ( a_idpt + b_x2 - 2.0 * c_musigmax ) / 2.0
print(exp, exp.shape)

[[-33.77546176 -41.24999765 -36.86827158 ... -31.61464305 -38.04546262
  -33.99665976]
 [-43.64193324 -61.33770892 -47.78282157 ... -30.16687173 -35.59531004
  -47.91293108]
 [-43.37894317 -43.17434871 -52.15635943 ... -53.28662124 -50.92538372
  -45.28740504]
 ...
 [-45.87834062 -42.45844774 -17.06297797 ... -32.69201927 -40.25855832
  -32.51538372]
 [-40.18901408 -32.70376534 -37.34189453 ... -43.02961482 -42.4241037
  -41.52696775]
 [-46.22356408 -43.75944722 -39.02603416 ... -45.33015984 -32.63540222
  -40.86911482]] (3288, 64)


## 1.2) Écrivez une fonction compute_lpp 

Cette méthode de la class **Mixture** va calculer les log posterior probabilités $$\log{p_i(x)}$$ d'un ensemble de vecteurs sur ce mélange de Gaussienne à matrices de covariances diagonales

Pour acumuler les statistiques, vous allez créer un objet mixture que vous appelerez **accum**.

* Cette méthode prend en paramètres une matrice de coefficients cepstraux de dimension N x F où N est le nombre de vecteurs (variable selon les fichiers) et F est la dimension des vecteurs (39 dans notre cas)
* En premier lieu, pensez à extraire des boucles tous les termes qui ne dépendent pas des données
* Cette méthode renvoie les $$n_i$$ (donc un vecteur)

Pour rappel: 
$$p_i(x) = \frac{1}{(2\pi)^{\frac{D}{2}}|\Sigma_i|^\frac{1}{2}} e^{-(\frac{1}{2}(x - \mu)^T\Sigma_i^{-1}(x-\mu))}$$
sachant:
$$ -(\frac{1}{2}(x - \mu_i)^T\Sigma_i^{-1}(x-\mu_i)) = -\frac{1}{2} ( x^T\Sigma_i^{-1}x + \mu_i^T\Sigma_i^{-1}\mu_i - x^T\Sigma_i^{-1}\mu_i - \mu_i^T\Sigma_i^{-1}x ) $$ 


In [2]:
import numpy
X = numpy.load("features_1.npy")

def compute_lpp(X):
    log_pi = numpy.log(mixture.cst)
    
    # Calcul de la partie indépendante des données:
    a_idpt = numpy.sum( numpy.square(mixture.mu) * mixture.invcov, axis=1 )
    
    # Calcul des parties dépendantes des données:
    b_x2 = []
    for x in X:
        b_x2.append(numpy.sum( numpy.square(x) * mixture.invcov, axis=1 ))
    b_x2 = numpy.array(b_x2)
    
    c_musigmax = []
    for x in X:
        c_musigmax.append(numpy.sum(x * mixture.invcov * mixture.mu, axis=1))
    c_musigmax = numpy.array(c_musigmax)
    
    exp = - ( a_idpt + b_x2 - 2.0 * c_musigmax ) / 2.0
    
    log_pi += exp
    
    return log_pi

(338, 39)

## 1.3) Utilisez la fonction **sum_log_probabilities**
* Que fait cette fonction qui vous est donnée?
* Pourquoi l'utilise-t'on?

In [2]:
def sum_log_probabilities(lp):
    """Sum log probabilities in a secure manner to avoid extreme values

    :param lp: numpy array of log-probabilities to sum
    """
    pp_max = numpy.max(lp, axis=1)
    log_lk = pp_max + numpy.log(numpy.sum(numpy.exp((lp.transpose() - pp_max).T), axis=1))
    ind = ~numpy.isfinite(pp_max)
    if sum(ind) != 0:
        log_lk[ind] = pp_max[ind]
    pp = numpy.exp((lp.transpose() - log_lk).transpose())
    llk = log_lk.sum()
    return pp, llk

## 1.4) Bouclez sur les fichiers de paramètres pour accumuler les statistiques